# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

### 1. What test is appropriate for this problem? Does CLT apply?

**Chi-square test** on independence is approriate for this problem. The problem here is to do statistical analysis to test whether race has a significant impact on the rate of callbacks for resumes. This is equivalent to the problem testing whether two variables `race` and `call` are independent. And also it is equivalent to the problem testing whether difference between the proportions of `call` in two `race` groups (b and w) is zero.

**Two-sample test** using bootstrap is also approriate for this problem.

As mentioned in the background, the résumés are randomly assigned to black-sounding or white-sounding names for the experience. Therefore, this is a random sampling. Then CLT can apply.

### 2. What are the null and alternate hypotheses?

Set $\delta$ as the difference between the proportions of `call` in two `race` groups (b and w). Then the hypothesis is:
$$ H_0: \delta = 0 $$
$$ H_a: \delta \neq 0 $$

### 3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

In [56]:
# Frequentist approach. Chi-square test on independence

# construct a 2-way contingency table
tbl_2way = pd.pivot_table(data, values='id', index='race', columns='call', aggfunc='count', margins=True) 
tbl_2way

call,0.0,1.0,All
race,,,
b,2278,157,2435
w,2200,235,2435
All,4478,392,4870


In [75]:
# Chi-square test
obs = np.array(tbl_2way.iloc[[0,1], [0,1]])
x, p, dof, expctd = stats.chi2_contingency(obs)
print('The p-value of Chi-test on independence is {}.'.format(p))

The p-value of Chi-test on independence is 4.997578389963255e-05.


In [117]:
# permutation approach for two-sample test

# concatnate two data and do permutation, then seperate them to two new data
def permutation_sample(data1, data2):
    """Generate a permutation sample from two data sets."""

    # Concatenate the data sets: data
    data = np.concatenate((data1, data2))

    # Permute the concatenated array: permuted_data
    permuted_data = np.random.permutation(data)

    # Split the permuted array into two: perm_sample_1, perm_sample_2
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]

    return perm_sample_1, perm_sample_2

def draw_perm_reps(data_1, data_2, func, size=1):
    """Generate multiple permutation replicates."""

    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty(size)

    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data_1, data_2)

        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)

    return perm_replicates
    
def diff_of_means(data_1, data_2):
    """Difference in means of two arrays."""

    # The difference of means of data_1, data_2: diff
    diff = np.mean(data_1) - np.mean(data_2)

    return diff

# two groups of data
b = data[data.race=='b']['call']
w = data[data.race=='w']['call']

# Draw 10,000 permutation replicates: perm_replicates
perm_replicates = draw_perm_reps(b, w, diff_of_means, size=10000)

# Compute difference of mean (proportion of call) between two groups
empirical_diff_means = np.mean(b) - np.mean(w)

# Compute p-value: p
p = np.sum(perm_replicates <= empirical_diff_means) / len(perm_replicates)

# Print the result
print('p-value using bootstrap is ', p)

# Compute 95% confidence interval
conf_int = np.percentile(perm_replicates, [2.5, 97.5])
print('95% CI using bootstrap is', conf_int)

p-value using bootstrap is  0.0
95% CI using bootstrap is [-0.01560576  0.01478439]


In [76]:
# computer %95 CI for difference
def diffprop(obs):
    """
    `obs` must be a 2x2 numpy array.

    Returns:
    delta
        The difference in proportions
    ci
        The Wald 95% confidence interval for delta
    corrected_ci
        Yates continuity correction for the 95% confidence interval of delta.
    """
    n1, n2 = obs.sum(axis=1)
    prop1 = obs[0,0] / n1
    prop2 = obs[1,0] / n2
    delta = prop1 - prop2

    # Wald 95% confidence interval for delta
    se = np.sqrt(prop1*(1 - prop1)/n1 + prop2*(1 - prop2)/n2)
    ci = (delta - 1.96*se, delta + 1.96*se)

    # Yates continuity correction for confidence interval of delta
    correction = 0.5*(1/n1 + 1/n2)
    corrected_ci = (ci[0] - correction, ci[1] + correction)

    return delta, ci, corrected_ci

diff, ci, corrected_ci = diffprop(obs)
print('The difference between proportions of two races is ', diff)
print('The %95 CI of difference between proportions of two races is', ci)

The difference between proportions of two races is  0.03203285420944568
The %95 CI of difference between proportions of two races is (0.016777447859559244, 0.04728826055933212)


In [118]:
rho_all = np.mean(data.call) # proportion of calls for all resumes
rho_b = np.mean(b) # proportion of calls in group b
rho_w = np.mean(w) # proportion of calls in group w

print(rho_all)
print(rho_b)
print(rho_w)

0.08049281686544418
0.0644763857126236
0.09650924056768417


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

### 4. Write a story describing the statistical significance in the context or the original problem.

The results of this study shows that 9.65% of the resumes with white-sounding names received callbacks, whereas only 6.45% of the resumes with black-sounding names received callbacks. The difference in their callback rates, 3.20%, is statistically significant (which means the difference is different from zero), with a Chi-square test of p-value close to 0. This suggests that, all other things being equal, race still has a statistically significant impact on the rate of callbacks the candidates receive. When given a random resume with either a black-sounding or white-sounding name, employers were more likely to call a white-sounding name for an interview.

### 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

The above analysis shows that race/name is indeed an important factor in callback success, but this does not necessarily mean race/name is the most important factor. This study only looked at the difference between race and call-back rate but there is a lot more than just someone's name on their resume that could also play a part in whether they get a call-back. 